In [ ]:
import tvm
from tvm import testing
from tvm import te
import numpy as np

target = "llvm"
target_host = "llvm"

N = te.var('n')
A = te.placeholder((N,),name='a')
B = te.placeholder((N,),name='b')

In [ ]:
A
A.shape
# A.dtype

In [ ]:
C = te.compute(A.shape,lambda i:A[i]+B[i],name='c')

In [ ]:
type(C)
# print(type(A),type(C))
C.op

In [ ]:
sche = te.create_schedule(C.op)

bx,tx = sche[C].split(C.op.axis[0], factor=64)

# sche[C].bind(bx,te.thread_axis("blockIdx.x"))
# sche[C].bind(tx,te.thread_axis("threadIdx.x"))

sche[C].parallel(bx)

In [ ]:
tadd = tvm.build(sche,[A,B,C],target=target,target_host=target_host,name="tadd")

In [ ]:
type(tadd)

In [ ]:
ctx = tvm.cpu(0)

n = 10000

In [ ]:
a = tvm.nd.array(np.random.uniform(size=n).astype(A.dtype),ctx)
b = tvm.nd.array(np.random.uniform(size=n).astype(B.dtype),ctx)
c = tvm.nd.array(np.zeros(n, dtype=C.dtype), ctx)


In [ ]:
tadd(a, b, c)

In [ ]:
c

In [ ]:
tvm.testing.assert_allclose(c.asnumpy(), a.asnumpy() + b.asnumpy())

In [ ]:
tadd.__dict__

In [ ]:
# tadd.imported_modules


tadd.get_source()

In [ ]:
import tvm
from tvm import testing
from tvm import te
import numpy as np

N= te.var('N')
A = te.placeholder((N,),name='A')
B = te.reduce_axis((0,N),name='B')

C = te.compute((1,),lambda i: te.sum(A[B], axis=B),name='C')

s = te.create_schedule(C.op)

print(type(N),type(A),type(B),type(C),type(s))

tvm.lower(s, [A,C],simple_mode=True)

In [ ]:
k0,k1 = s[C].split(C.op.reduce_axis[0], factor=32)

tvm.lower(s,[A,C],simple_mode=True)

In [ ]:
# k0,k1 = s[C].split(C.op.reduce_axis[0], factor=32)

s[C].parallel(k0)

tvm.lower(s,[A,C],simple_mode=True)

In [ ]:
import tvm
from tvm import testing
from tvm import te
import numpy as np

target = "llvm"
target_host = "llvm"

N = te.var('n')
A = te.placeholder((N,N),name='a')
B = te.placeholder((N,N),name='b')

C = te.compute((N,N),lambda i,j : A[i,j] + B[i,j],name='c')
s = te.create_schedule(C.op)

print(tvm.lower(s, [A,B,C], simple_mode=True))

x1,x2 = s[C].split(C.op.axis[0], factor=4)
y1,y2 = s[C].split(C.op.axis[1], factor=4)

s[C].reorder(x1, y1,x2,y2)

print(tvm.lower(s, [A,B,C], simple_mode=True))

In [ ]:
import tvm
from tvm import testing
from tvm import te
import numpy as np

target = "llvm"
target_host = "llvm"

N = te.var('n')
A = te.placeholder((N,N),name='a')
B = te.placeholder((N,N),name='b')

C = te.compute((N,N),lambda i,j : A[i,j] + B[i,j],name='c')
s = te.create_schedule(C.op)

print(tvm.lower(s, [A,B,C], simple_mode=True))


print(tvm.lower(s, [A,B,C], simple_mode=True))

In [ ]:
import tvm
from tvm import testing
from tvm import te
import numpy as np

target = "llvm"
target_host = "llvm"

N = te.var('n')
A = te.placeholder((N,N),name='a')
B = te.placeholder((N,N),name='b')

C = te.compute((N,N),lambda i,j : A[i,j] + B[i,j],name='c')
s = te.create_schedule(C.op)

xo, yo, xi, yi = s[C].tile(C.op.axis[0], C.op.axis[1],x_factor=32, y_factor=32)

print(tvm.lower(s, [A,B,C], simple_mode=True))

s[C].vectorize(yi)

print(tvm.lower(s, [A,B,C], simple_mode=True))